### Block particle filter

* Algorithm 2 of https://projecteuclid.org/download/pdfview_1/euclid.aoap/1438261054

In [1]:
# Calling libraries:
from __future__ import division
%matplotlib inline
import numpy as np, time, matplotlib.pyplot as plt, math, pandas, numpy.random as npr, multiprocessing as mp, timeit
from pylab import plot, show, legend
from time import time
from scipy.stats import *
from tqdm import trange
from ecology_functions import *

In [2]:
T = 100
I = 20  # number of locations
J = 10  # number of species
K = 3   # number of latent factors

In [3]:
lmbda = npr.randn(J,K)
alpha = npr.randn(J)
c = 0
phi = 0.5
logsigmasq = 0
x_0 = npr.randn(I,K)

theta = [alpha, lmbda, c, phi, logsigmasq]
Y, X = simulate_data(x_0, T, J, theta)

In [4]:
n_particles = 1000
%timeit block_pf(Y, x_0, n_particles, theta, calc_grad=True)
%timeit bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=True)
%timeit block_pf(Y, x_0, n_particles, theta, calc_grad=False)
%timeit bootstrap_PF_grad(x_0, n_particles, theta, Y, calc_grad=False)

6.53 s ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.17 s ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.76 s ± 5.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.36 s ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
n_particles = 1000
logNC_block, grad_block, particles_block, weights_block = block_pf(Y, x_0, n_particles, theta, True)
print(logNC_block)
logNC_bootstrap, grad_bootstrap, particles_bootstrap, weights_bootstrap = bootstrap_PF_grad(x_0, n_particles, theta, Y, True)
print(logNC_bootstrap)

-16504.68585831285
-18708.85587038649


In [5]:
grad_block

[array([ -704., -1321., -1042., -1512.,  -879., -1535., -1452.,  -938.,
         -623.,  -854.]), array([[  81.83952581,  214.99138149,  589.57388502],
        [-104.58695547,  639.83691002, -294.11889498],
        [ -28.99890473, -153.59321965,  701.01876454],
        [ 560.78993726,  937.20684772,   44.33712458],
        [   1.17790211,  793.16729409, -130.88770915],
        [ 406.62161582,  578.4163835 ,  630.16276757],
        [ 726.3860151 , -409.71724239, -274.53718736],
        [ 881.62756408, -429.02111244,  745.79827221],
        [-157.20737258,  985.3525858 , -232.83810557],
        [ 420.57645478, -840.8888203 ,   57.01625507]]), 0.0, 0.0, 0.0]

In [10]:
grad_bootstrap

[array([ -704., -1321., -1042., -1512.,  -879., -1535., -1452.,  -938.,
         -623.,  -854.]), array([[ 59.03782232,  64.1144089 ,  38.1954198 ],
        [  5.08636904, 204.14648824,  50.49466052],
        [ 24.60335159,  35.47850043,  90.9019978 ],
        [ 30.45487034, 207.06583077,  35.2821975 ],
        [ -4.55613095, 161.56059974, -35.32591573],
        [ 63.15568313, 113.0865661 ,  73.45836583],
        [111.91204374, 113.0088321 ,  68.02770738],
        [113.14477397,   3.34431378,  77.75351234],
        [-22.18617403, 100.02908184,  19.63307632],
        [131.68576961,  -4.08541256,  46.55516131]]), -212.19556806898453, -5435.6693100287375, 2855.6573347811077]

In [9]:
np.sum(np.abs(grad_bootstrap[0]-grad_block[0])), np.sum(np.abs(grad_bootstrap[1]-grad_block[1]))

(8.185452315956354e-12, 11593.387857705353)

In [10]:
logNC

-85.98063988535276

In [6]:
n_particles = 1000
particles = npr.randn(n_particles,I,K) 
propagated_particles = npr.randn(*np.shape(particles))
y = Y[1]

In [7]:
%timeit alpha_grad_blockPF(y, theta, propagated_particles, particles)
%timeit lmbda_grad_blockPF(y, theta, propagated_particles, particles)
%timeit c_grad_blockPF(y, theta, propagated_particles, particles)
%timeit phi_grad_blockPF(y, theta, propagated_particles, particles)
%timeit logsigmasq_grad_blockPF(y, theta, propagated_particles, particles)

131 µs ± 250 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
2.43 ms ± 7.05 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
240 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
303 µs ± 992 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
269 µs ± 688 ns per loop (mean ± std. dev. of 7 runs, 1000 loops each)
